In [ ]:
# LLM 모델을 통한 연산 예제
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)
# llm = ChatOpenAI(model_name='gpt-4o',temperature=0)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12."

llm.invoke(prompt).content

In [13]:
# LLM 모델을 통한 연산 예제
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0)

# 덧셈을 수행하는 함수 정의
def plus(a, b): 
    return a + b

# 에이전트 생성
agent = initialize_agent(
    # 사용할 LLM 모델
    llm=llm,

    # 추론 과정 디버깅(에전트가 하는 작업의 모든 과정 및 추론을 직접 확인해볼 수 있음)
    verbose = True, 

    # 에이전트 유형 설정
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,

    # 에이전트가 사용할 도구 모음(Tools)
    tools=[
        StructuredTool.from_function(
            func = plus, # 위에서 정의해놓은 덧셈 함수를 도구 모음에 추가
            name = "Sum Calculator", # 도구의 이름을 "덧셈 계산기"로 정의 
            description="Use this to perform sums of two numbers. This tool take two arguments, both should be numbers. till finish." # 도구에 대한 설명 작성
        )
    ]
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12."


# 이번엔 에이전트에게 프롬프트를 입력(위에선 LLM모델에 입력했음)
agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the total cost of all the individual amounts provided by the user.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 355.39, "b": 924.87}
}
```

Observation: 1280.26
Thought:I have calculated the sum of the first two amounts correctly. Now, I will continue to calculate the sum of the remaining amounts provided by the user.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 1280.26, "b": 721.2}
}
```
Observation: 2001.46
Thought:I have calculated the sum of the first three amounts correctly. Now, I will continue to calculate the sum of the remaining amounts provided by the user.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 2001.46, "b": 1940.29}
}
```
Observation: 3941.75
Thought:I have calculated the sum of the first four amounts correctly. Now, I will continue to calculate the sum of the remaining amounts provided by the user.
Action:
```

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12.',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

In [ ]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)